# Домашнее задание №2 по теме "Бейзлайны и детерминированные алгоритмы item-item"

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


Возьмем последние 3 недели в качестве теста

In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

In [6]:
%%time

6534178

items_weights = data_train.groupby('item_id')['quantity'].sum().reset_index(name='weight')
n_interactions = items_weights['weight'].sum()
items_weights['weight'] = items_weights['weight']/n_interactions

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: total: 2.98 s
Wall time: 2.99 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6533889, 6534178, 1404121, 6534166, 397896]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6533889, 6534166, 420647, 6534178, 480014]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

#### 1 Random recommendation

In [7]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [8]:
items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6533889, 6534178, 1404121, 6534166, 397896]","[833064, 9796639, 1029894, 12263026, 6391544]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6533889, 6534166, 420647, 6534178, 480014]","[9855017, 1111493, 6703940, 826603, 10150063]"


#### 2 Popularity-based recommendation

In [9]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [10]:
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6533889, 6534178, 1404121, 6534166, 397896]","[833064, 9796639, 1029894, 12263026, 6391544]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6533889, 6534166, 420647, 6534178, 480014]","[9855017, 1111493, 6703940, 826603, 10150063]","[6534178, 6533889, 1029743, 6534166, 1082185]"


#### 3 Weighted random recommender

Уже добавили в Задании №1

#### 4 Item-Item Recommender / ItemKNN

In [11]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

In [12]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [13]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix, чтобы использовать в user_items
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Для работы с матрицами заведем словари перевода идентификаторов в индексы матриц и обратно

In [14]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [15]:
model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)
recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, # нужно ли фильтровать товары, с которыми юзер уже взаимодействовал
                        filter_items=[itemid_to_id[999999]], # какие товары нам нужно отфильтровать
                        recalculate_user=False # делать ли пересчет алгоритма, когда мы делаем рекомендацию
                      ) 

  0%|          | 0/5001 [00:00<?, ?it/s]

In [16]:
result['itemitem_k=5'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],                                    
                                    recalculate_user=True)[0]])
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem_k=5
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6533889, 6534178, 1404121, 6534166, 397896]","[833064, 9796639, 1029894, 12263026, 6391544]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6533889, 6534166, 420647, 6534178, 480014]","[9855017, 1111493, 6703940, 826603, 10150063]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]"


#### 5 Косинусное сходство и CosineRecommender

In [17]:
model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей
model.fit(csr_matrix(user_item_matrix).tocsr(), 
          show_progress=True)
recs = model.recommend(userid=userid_to_id[2], 
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]],      
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

In [18]:
result['cosine_k=5'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],                                      
                                    recalculate_user=True)[0]])
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem_k=5,cosine_k=5
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6533889, 6534178, 1404121, 6534166, 397896]","[833064, 9796639, 1029894, 12263026, 6391544]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6533889, 6534166, 420647, 6534178, 480014]","[9855017, 1111493, 6703940, 826603, 10150063]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]"


#### 6 TF-IDF взвешивание и TFIDFRecommender

In [19]:
model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей
model.fit(csr_matrix(user_item_matrix).tocsr(), 
          show_progress=True)
recs = model.recommend(userid=userid_to_id[2], 
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False,
                        filter_items=[itemid_to_id[999999]],                        
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

In [20]:
result['tfidf_k=5'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],                                     
                                    recalculate_user=False)[0]])
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem_k=5,cosine_k=5,tfidf_k=5
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6533889, 6534178, 1404121, 6534166, 397896]","[833064, 9796639, 1029894, 12263026, 6391544]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6533889, 6534166, 420647, 6534178, 480014]","[9855017, 1111493, 6703940, 826603, 10150063]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]"


#### Расчет метрик

In [21]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [22]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list[:k])
    recall = flags.sum() / len(bought_list)
    
    return recall

In [23]:
evaluate_dict = {}

In [24]:
for alg_name in result.columns.drop(['user_id', 'actual']).tolist():
    evaluate_dict[alg_name] = []
    evaluate_dict[alg_name].append(result.apply(lambda row: precision_at_k(row[alg_name], row['actual'], k=5), axis=1).mean())
    evaluate_dict[alg_name].append(result.apply(lambda row: recall_at_k(row[alg_name], row['actual'], k=5), axis=1).mean())

In [25]:
evaluate_df = pd.DataFrame(evaluate_dict, index=['Precision@5', 'Recall@5'])
evaluate_df.T.sort_values('Precision@5', ascending=False)

,Precision@5,Recall@5
popular_recommendation,0.155240,0.024996
itemitem_k=5,0.145739,0.016218
cosine_k=5,0.135455,0.014031
tfidf_k=5,0.135455,0.014031
weighted_random_recommendation,0.047992,0.012198
random_recommendation,0.000294,0.000014


**Вывод:** Согласно полученным оценкам, наиболее качественным является алгоритм *Popularity-based recommendation*.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

Пробуем менять число соседей $K$ в детерминированных алгоритмах:

In [26]:
k_list = np.arange(1, 11, 1)

**Item-Item Recommender**

In [27]:
for k_i in k_list: 
    model = ItemItemRecommender(K=k_i, num_threads=4) 
    model.fit(sparse_user_item,
              show_progress=True)
    recs = model.recommend(userid=userid_to_id[2],
                           user_items=sparse_user_item,
                           N=5,
                           filter_already_liked_items=False,
                           filter_items=[itemid_to_id[999999]],
                           recalculate_user=False)
    result[f'itemitem_k={k_i}'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=[itemid_to_id[999999]],                                    
                                        recalculate_user=True)[0]])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

**CosineRecommender**

In [28]:
for k_i in k_list: 
    model = CosineRecommender(K=k_i, num_threads=4) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).tocsr(),
              show_progress=True)
    recs = model.recommend(userid=userid_to_id[2],
                            user_items=sparse_user_item,   # на вход user-item matrix
                            N=5,
                            filter_already_liked_items=False,
                            filter_items=[itemid_to_id[999999]],
                            recalculate_user=False)
    result[f'cosine_k={k_i}'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=[itemid_to_id[999999]],                                      
                                        recalculate_user=True)[0]])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

**TF-IDF**

In [29]:
for k_i in k_list: 
    model = TFIDFRecommender(K=k_i, num_threads=4) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).tocsr(), 
              show_progress=True)
    recs = model.recommend(userid=userid_to_id[2], 
                            user_items=sparse_user_item,   # на вход user-item matrix
                            N=5, 
                            filter_already_liked_items=False,
                            filter_items=[itemid_to_id[999999]],                        
                            recalculate_user=False)
    
    result[f'tfidf_k={k_i}'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=[itemid_to_id[999999]],                                     
                                        recalculate_user=False)[0]])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

**Оценка**

In [30]:
evaluate_dict = {}

In [31]:
for alg_name in result.columns.drop(['user_id', 'actual']).tolist():
    evaluate_dict[alg_name] = []
    evaluate_dict[alg_name].append(result.apply(lambda row: precision_at_k(row[alg_name], row['actual'], k=5), axis=1).mean())
    evaluate_dict[alg_name].append(result.apply(lambda row: recall_at_k(row[alg_name], row['actual'], k=5), axis=1).mean())

In [32]:
evaluate_df = pd.DataFrame(evaluate_dict, index=['Precision@5', 'Recall@5'])
evaluate_df.T.sort_values('Precision@5', ascending=False).head(10)

,Precision@5,Recall@5
itemitem_k=3,0.192850,0.022719
itemitem_k=2,0.162977,0.019229
itemitem_k=1,0.162292,0.018309
itemitem_k=10,0.157003,0.017730
itemitem_k=4,0.157003,0.017730
itemitem_k=6,0.157003,0.017730
itemitem_k=7,0.157003,0.017730
itemitem_k=8,0.157003,0.017730
itemitem_k=9,0.157003,0.017730
popular_recommendation,0.155240,0.024996


**Вывод:** Изменяя параметры бейзлайнов, удалось получить результат получше. Теперь наиболее оптимальным алгоритмом стал Item-Item с количеством соседей равным 3.

---